In [1]:
import sys
sys.path.append('models')

# from google.colab import drive
# drive.mount('/content/drive')
# sys.path.append('/content/drive/MyDrive/Dissertation/models')
# sys.path.append('/content/drive/MyDrive/Dissertation')

In [2]:
# !pip install dask_ml
# !pip install scikeras
# import nltk
# nltk.download('stopwords')
# import nltk
# nltk.download('punkt')

In [3]:
# importing libraries
import joblib
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from collections import defaultdict
import matplotlib.pyplot as plt

from logstic_regression import Logistic_Regression
from XGBoost import XGBoost
from naive_bayes import Naive_Bayes
from rnn import RNN
from cnn import CNN
# from models.bert import BERT
from bilstm import BiLSTM

# Load the TextPreprocessor class (assumed to be defined already)
from textpreprocessor import TextPreprocessor
from evaluation_visualization import Evaluation_Visualization

import warnings
warnings.filterwarnings("ignore")

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2024-10-26 23:49:25.110986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 23:49:25.119726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 23:49:25.122230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 23:49:25.128525: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 23:49:25.533481: W tensorflow/compiler/tf2

Num GPUs Available:  1


I0000 00:00:1729979366.634354  701133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729979366.640674  701133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729979366.640708  701133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
NUM_SAMPLE = 10000
TEST_RATIO=0.2
BATCH_SIZE=512
EPOCHS = 10
MAX_WORD_COUNT = 5000
MAX_LENGTH = 222
OUTPUT_RESULT_DIR = "output/result"
OUTPUT_MODELS_DIR = "output/models"
USE_TEST_DATA = True
INPUT_DIR = f"input/sample_{NUM_SAMPLE}"

os.makedirs(OUTPUT_RESULT_DIR, exist_ok=True)
os.makedirs(OUTPUT_MODELS_DIR, exist_ok=True)

eval_and_visual = Evaluation_Visualization(out_result_dir= OUTPUT_RESULT_DIR, out_models_dir=OUTPUT_MODELS_DIR)

00. Text Pre-Processing

In [5]:
# Initialize the Text Preprocessor
processor = TextPreprocessor(MAX_WORD_COUNT, MAX_LENGTH)

if USE_TEST_DATA:
    # Load data
    df_train = pd.read_csv(os.path.join(INPUT_DIR, 'train_cleaned.csv'))
    df_test = pd.read_csv(os.path.join(INPUT_DIR, 'test_cleaned.csv'))
    # df_test = processor.load_data()
    X_train = df_train['review']
    X_test = df_test['review']
    y_train = df_train['polarity']
    y_test = df_test['polarity']
    # X_train_seq_padded = pickle.load(os.path.join(INPUT_DIR, 'X_train_pad.pkl'))
    # X_test_seq_padded = pickle.load(os.path.join(INPUT_DIR, 'X_test_pad.pkl'))
else:
    # Load data
    df_train, df_test = processor.parallel_load_data()

    df_train_step1 = processor.remove_stopwords(df_train.copy())
    df_test_step1 = processor.remove_stopwords(df_test.copy())

    print('----------TRAIN DATA----------')
    df_train_step2 = processor.filter_by_length_of_sentence(df_train_step1.copy(),50)
    print('----------TEST DATA----------')
    df_test_step2 = processor.filter_by_length_of_sentence(df_test_step1.copy(),50)

    df_train_step3 = processor.sampling_data(df_train_step2, NUM_SAMPLE)
    df_test_step3 = processor.sampling_data(df_test_step2, int(NUM_SAMPLE*TEST_RATIO))

    # Preprocess data
    df_train_step3 = processor.map_polarity(df_train_step3.copy())
    df_test_step3 = processor.map_polarity(df_test_step3.copy())

    # Split data
    X_train, y_train = processor.split_data(df_train_step3)
    X_test, y_test = processor.split_data(df_test_step3)
    
    os.makedirs(INPUT_DIR, exist_ok=True)
    # Save data
    df_train_step3.to_csv(os.path.join(INPUT_DIR, 'train_cleaned.csv'), index=False)
    df_test_step3.to_csv(os.path.join(INPUT_DIR, 'test_cleaned.csv'), index=False)

X_train_tfidf, X_test_tfidf = processor.vectorize_text(X_train, X_test)
X_train_pad, X_test_pad = processor.tokenization_and_padding(X_train, X_test)

MAX_LENGTH = processor.max_length

02. XGBoost

In [6]:
# 1. Train Model
xgboost = XGBoost(verbose=1)
xgboost.train_model(X_train_tfidf.toarray(), y_train)


In [10]:

# 2. Random SearchCV
xgboost.random_search(X_train_tfidf.toarray(), y_train, n_iter=2, cv=2, random_state=42, n_jobs=-1)

_, best_params = eval_and_visual.compare_models_accuracy_and_get_best_params({'Original': xgboost.model,
                                                              'RandomizedSearchCV': xgboost.random_search_cv}, X_test_tfidf.toarray(), y_test)


[09:10:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[09:10:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[09:10:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[09:10:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[09:10:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



Best parameters found: {'use_label_encoder': False, 'tree_method': 'hist', 'subsample': 0.6, 'n_estimators': None, 'min_child_weight': None, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0.0, 'eval_metric': 'logloss', 'device': 'cuda', 'colsample_bytree': 0.6}
Original Accuracy: 71.5
RandomizedSearchCV Accuracy: 78.25
Best Model: RandomizedSearchCV with Accuracy: 78.25
Best Parameters: {'use_label_encoder': False, 'tree_method': 'hist', 'subsample': 0.6, 'n_estimators': None, 'min_child_weight': None, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0.0, 'eval_metric': 'logloss', 'device': 'cuda', 'colsample_bytree': 0.6}


Best parameters found: {'use_label_encoder': False, 'tree_method': 'hist', 'subsample': None, 'n_estimators': None, 'min_child_weight': 2, 'max_depth': 7, 'learning_rate': None, 'gamma': 0.15, 'eval_metric': 'logloss', 'device': 'cuda', 'colsample_bytree': None}
<br>
Original Accuracy: 71.5
<br>
RandomizedSearchCV Accuracy: 78.85
<br>
Best Model: RandomizedSearchCV with Accuracy: 78.85
<br>
Best Parameters: {'use_label_encoder': False, 'tree_method': 'hist', 'subsample': None, 'n_estimators': None, 'min_child_weight': 2, 'max_depth': 7, 'learning_rate': None, 'gamma': 0.15, 'eval_metric': 'logloss', 'device': 'cuda', 'colsample_bytree': None}

In [9]:
best_params = {
    'colsample_bytree': 0.5499999999999999, 
    'device': 'cuda', 
    'eval_metric': 'logloss', 
    'gamma': 0.2, 
    'learning_rate': 0.02, 
    'min_child_weight': 2, 
    'n_estimators': 100, 
    'subsample': 0.11, 
    'tree_method': 'hist', 
    'use_label_encoder': False
    }

In [7]:
best_params = {
    'use_label_encoder': False, 
    'tree_method': 'hist', 
    'subsample': None, 
    'n_estimators': None, 
    'min_child_weight': 2, 
    'max_depth': 7, 
    'learning_rate': None, 
    'gamma': 0.15, 
    'eval_metric': 'logloss', 
    'device': 'cuda', 
    'colsample_bytree': None}


In [8]:

# 3. Grid SearchCV
xgboost.grid_search(X_train_tfidf.toarray(), y_train, cv=5,  n_jobs=-1, best_params=best_params)

# 4. Train Best Model
xgboost.train_best_model(X_train_tfidf.toarray(), y_train, xgboost.grid_search_cv.best_params_)


[23:50:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:50:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_la

Best parameters found by GridSearchCV: {'colsample_bytree': None, 'device': 'cuda', 'eval_metric': 'logloss', 'gamma': 0.2, 'learning_rate': None, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': None, 'subsample': None, 'tree_method': 'hist', 'use_label_encoder': False}


In [11]:

# 5. Evaluate and Save Models
eval_and_visual.evaluate_model_class(xgboost, X_test_tfidf.toarray(), y_test, file_name="XGBoost_jupyter.xlsx")

xgboost.save_model_and_params(
    os.path.join(OUTPUT_MODELS_DIR, 'xgboost_model.pkl'),
    os.path.join(OUTPUT_MODELS_DIR, 'xgboost_best_model.pkl'),
    os.path.join(OUTPUT_MODELS_DIR, 'xgboost_best_params.pkl')
    )


Model saved to Output/models/xgboost_model.pkl
Best model saved to Output/models/xgboost_best_model.pkl
Best parameters saved to Output/models/xgboost_best_params.pkl


In [12]:
eval_and_visual.df_results

,Model,Training-Time,Accuracy,Precision (Class 0),Precision (Class 1),Recall (Class 0),Recall (Class 1),F1-Score (Class 0),F1-Score (Class 1)
0,XGBoost,3.079637,71.50,0.654842,0.780564,0.764838,0.674797,0.705579,0.723837
1,XGBoost_random_search,3.025182,78.25,0.746767,0.813433,0.776036,0.787715,0.761120,0.800367
2,XGBoost_grid_search,24479.923968,79.75,0.765217,0.825000,0.788354,0.804878,0.776613,0.814815
3,XGBoost_best,1.178377,79.75,0.765217,0.825000,0.788354,0.804878,0.776613,0.814815
